In [1]:
from sqlalchemy import *
import numpy as np
import pylab as plt
import pandas as pd
from scipy.ndimage.interpolation import shift
from matplotlib.colors import LogNorm
from scipy.interpolate import interp1d
from scipy.optimize import newton
import cv2
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import plotly
plotly.offline.init_notebook_mode()
import plotly.plotly as py
from plotly.offline import iplot,plot
import plotly.graph_objs as go
import time
import re
import math
import seaborn as sns

from skimage.transform import (hough_line, hough_line_peaks,
                               probabilistic_hough_line)
import heapq
%matplotlib inline

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [2]:
start_time = time.time()
engine = create_engine('sqlite+pysqlite:///../../data/SQL/10C/85_8.db')
ADCdf = pd.io.sql.read_sql("SELECT * FROM ADC limit 100", engine)
end_time = time.time()
print end_time-start_time

engine = create_engine('sqlite+pysqlite:///../../data/SQL/10C/ProtoMap.db')
ProtoMapdf = pd.io.sql.read_sql("SELECT * FROM ProtoMap", engine)
print ProtoMapdf.head()

0.0866298675537
   PadNum        PadName     PadX      PadY  BinNum
0       0  sector_center  0.00000  0.000000     257
1       1      sector1_0  1.30108  0.947523       1
2       2      sector2_0  2.18496  1.831410       2
3       3      sector1_1  2.71529  2.361740       3
4       4      sector2_1  3.59917  3.245620       4


In [3]:
ADCdf.columns = [np.uint16(_[1:]) if re.match('t\d+',_) is not None else str(_) for _ in ADCdf.columns ]
#ADCdf['max'] = ADCdf.iloc[:,3+50:-50].max(axis=1)
#mask= (ADCdf.iloc[:,3:-1]>20) & (ADCdf.iloc[:,3:-1].gt(ADCdf['max']*0.2,axis=0))
#ADCdf.iloc[:,3:-1] = ADCdf.iloc[:,3:-1][mask].fillna(0)
#end_time = time.time()


In [4]:
ADCdf.loc['max'] = ADCdf.iloc[:,3+50:-50].max(axis=1)

In [5]:
ADCdf.head()

,ID,EventID,PadNum,0,1,2,3,4,5,6,...,502,503,504,505,506,507,508,509,510,511
0,1.0,0.0,97.0,-16.200001,-0.2,-1.2,1.8,0.8,-4.2,3.8,...,-3.2,-3.2,-5.2,-5.2,-5.2,-8.2,-6.2,-8.2,-1.2,-5.2
1,2.0,0.0,158.0,-8.300000,-1.3,-1.3,0.7,-1.3,-4.3,6.7,...,1.7,2.7,3.7,3.7,4.7,-1.3,-1.3,1.7,7.7,5.7
2,3.0,0.0,161.0,-8.500000,-0.5,2.5,4.5,1.5,-8.5,2.5,...,-1.5,-1.5,1.5,-0.5,2.5,1.5,4.5,4.5,9.5,3.5
3,4.0,0.0,95.0,-7.600000,-5.6,-4.6,0.4,1.4,0.4,4.4,...,-0.6,1.4,-1.6,3.4,1.4,0.4,-2.6,-2.6,8.4,4.4
4,5.0,0.0,98.0,-5.800000,-0.8,-0.8,3.2,0.2,-4.8,1.2,...,2.2,-1.8,0.2,3.2,3.2,3.2,1.2,2.2,6.2,4.2
